Dynamic programming refers to collection of algorithms that can be used to compute the optimal policies given a perfect model of the environment as MDP(Markov Decision Process)

Limitations of DP , because of the assumptions of <b>perfect model</b> and <b>great computionally expensive</b>

<b>The Key idea of DP is to use the value functions to organize and structure the search for good policies </b>

As we already know, we can get the optimal policies once we have found the optimal value functions

\begin{align}
V(s) = \sum_a\pi(a|s)\sum_{s^\prime,r}p(s^\prime, r|s, a)(r+\gamma*V(s^\prime)))
\end{align}

\begin{align}
Q(s, a) = \sum_{s^\prime,r}p(s^\prime, r|s, a)(r+\gamma*\sum_{a^\prime}\pi(a^\prime|s^\prime)Q(s^\prime,a^\prime)))
\end{align}

### Policy Evaluation:
Computing state value function for arbitrary policy is called policy evaluation or prediction problem
<br>
<br>

<img src='images/policy_evaluation.png' width=600>

In [53]:
import gym
import numpy as np
env = gym.make('FrozenLake-v0')

In [54]:
print("Number of States :", env.observation_space)

print("Number of Actions : ", env.action_space)

Number of States : Discrete(16)
Number of Actions :  Discrete(4)


In [55]:
env.render()


SFFF
FHFH
FFFH
HFFG


In [56]:
env.P[0]

{0: [(0.3333333333333333, 0, 0.0, False),
  (0.3333333333333333, 0, 0.0, False),
  (0.3333333333333333, 4, 0.0, False)],
 1: [(0.3333333333333333, 0, 0.0, False),
  (0.3333333333333333, 4, 0.0, False),
  (0.3333333333333333, 1, 0.0, False)],
 2: [(0.3333333333333333, 4, 0.0, False),
  (0.3333333333333333, 1, 0.0, False),
  (0.3333333333333333, 0, 0.0, False)],
 3: [(0.3333333333333333, 1, 0.0, False),
  (0.3333333333333333, 0, 0.0, False),
  (0.3333333333333333, 0, 0.0, False)]}

#### Actions

        LEFT = 0
        DOWN = 1
        RIGHT = 2
        UP = 3 

In [57]:
def iterative_policy_evaluation(env, policy, gamma=1.0, theta=1e-6):
    '''
    Calcuates State value function
    Inputs: 
        env - Environment 
        policy - Random Policy which follows by agent to calculate state value funtion
        gamma - gamma factor for discounted return
        theta - threshold for determing the accuracy of the estimation
    
    '''
    
    V = np.zeros(env.nS) ## initialize all states to zero
    while True:
        delta = 0 
        for state in range(env.nS):
            v_s = 0 
            for action, action_prob in enumerate(policy[state]):
                for prob, next_state, reward, done in env.P[state][action]:
                    v_s += action_prob * prob * (reward + gamma * V[next_state])
            delta = max(delta, abs(V[state] - v_s))
            V[state] = v_s
        if delta<theta:
            break
    return V
    

In [58]:
random_policy = np.ones([env.nS, env.nA]) / env.nA

In [59]:
V = iterative_policy_evaluation(env, random_policy)

In [60]:
env.render()


SFFF
FHFH
FFFH
HFFG


In [61]:
print("State-Value Function:")
print(V.reshape(4, 4))

State-Value Function:
[[0.0139372  0.01162942 0.02095187 0.01047569]
 [0.01624741 0.         0.04075119 0.        ]
 [0.03480561 0.08816967 0.14205297 0.        ]
 [0.         0.17582021 0.43929104 0.        ]]


In [62]:
def action_value_function(env, V, gamma=1):
    '''
    Input:
        env:
        V:
        gamma:
    
    Output:
        Q
    '''
    Q = np.zeros([env.nS, env.nA])
    for state in range(env.nS):
        for action in range(env.nA):
            for prob, next_state, reward, done in env.P[state][action]:
                Q[state][action] += prob * (reward + gamma * V[next_state])
    return Q

In [63]:
Q = action_value_function(env, V)
print("Action-Value Function:")
print(Q)

Action-Value Function:
[[0.01470727 0.01393801 0.01393801 0.01316794]
 [0.00852221 0.01162969 0.01086043 0.01550616]
 [0.02444416 0.0209521  0.02405958 0.01435233]
 [0.01047585 0.01047585 0.00698379 0.01396775]
 [0.02166341 0.01701767 0.0162476  0.01006154]
 [0.         0.         0.         0.        ]
 [0.05433495 0.04735099 0.05433495 0.00698396]
 [0.         0.         0.         0.        ]
 [0.01701767 0.04099176 0.03480569 0.04640756]
 [0.0702086  0.11755959 0.10595772 0.05895286]
 [0.18940397 0.17582024 0.16001408 0.04297362]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.08799662 0.20503708 0.23442697 0.17582024]
 [0.25238807 0.53837042 0.52711467 0.43929106]
 [0.         0.         0.         0.        ]]


#### Policy Improvement:
Given a value function corresponding to a policy, proposes a better or equal policy.
we calculate action value function from the state value function and pick the action which maximizes the action value function

\begin{align}
v_{\pi^\prime}(s) = \max_a\sum_{s^\prime,r}p(s^\prime, r|s, a)(r+\gamma*v_{\pi^\prime}(s^\prime))
\end{align}


In [64]:
def policy_improvement(env, V, gamma=1.0):
    '''
    Input :
        env
        V
        gamma
    Output:
        policy
    '''
    policy = np.zeros([env.nS, env.nA])
    Q = action_value_function(env, V, gamma)
    
    for state in range(env.nS):
        best_actions = np.argwhere(Q[state] == max(Q[state])).flatten()
        policy[state] = [1/len(best_actions) if i in best_actions else 0 for i in range(env.nA)]
        
    return policy

In [65]:
Q = action_value_function(env, V, 1.0)
Q

array([[0.01470727, 0.01393801, 0.01393801, 0.01316794],
       [0.00852221, 0.01162969, 0.01086043, 0.01550616],
       [0.02444416, 0.0209521 , 0.02405958, 0.01435233],
       [0.01047585, 0.01047585, 0.00698379, 0.01396775],
       [0.02166341, 0.01701767, 0.0162476 , 0.01006154],
       [0.        , 0.        , 0.        , 0.        ],
       [0.05433495, 0.04735099, 0.05433495, 0.00698396],
       [0.        , 0.        , 0.        , 0.        ],
       [0.01701767, 0.04099176, 0.03480569, 0.04640756],
       [0.0702086 , 0.11755959, 0.10595772, 0.05895286],
       [0.18940397, 0.17582024, 0.16001408, 0.04297362],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.08799662, 0.20503708, 0.23442697, 0.17582024],
       [0.25238807, 0.53837042, 0.52711467, 0.43929106],
       [0.        , 0.        , 0.        , 0.        ]])

In [66]:
env.render()


SFFF
FHFH
FFFH
HFFG


In [67]:
policy_improvement(env, V)

array([[1.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 1.  ],
       [1.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 1.  ],
       [1.  , 0.  , 0.  , 0.  ],
       [0.25, 0.25, 0.25, 0.25],
       [0.5 , 0.  , 0.5 , 0.  ],
       [0.25, 0.25, 0.25, 0.25],
       [0.  , 0.  , 0.  , 1.  ],
       [0.  , 1.  , 0.  , 0.  ],
       [1.  , 0.  , 0.  , 0.  ],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.  , 0.  , 1.  , 0.  ],
       [0.  , 1.  , 0.  , 0.  ],
       [0.25, 0.25, 0.25, 0.25]])

#### Policy Iteration

In [68]:
def policy_iteration(env, gamma=0.9, theta=1e-8):
    
    policy = np.ones([env.nS, env.nA]) / env.nA
    
    for _ in range(100):
        V = iterative_policy_evaluation(env, policy, gamma, theta)
        new_policy = policy_improvement(env, V, gamma)
        
        if (new_policy == policy).all():
            break
        policy = np.copy(new_policy)
        
    return policy, V

In [69]:
# Truncated Policy Iteration

In [70]:
policy, V = policy_iteration(env)

In [71]:
policy

array([[1.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 1.  ],
       [1.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 1.  ],
       [1.  , 0.  , 0.  , 0.  ],
       [0.25, 0.25, 0.25, 0.25],
       [0.5 , 0.  , 0.5 , 0.  ],
       [0.25, 0.25, 0.25, 0.25],
       [0.  , 0.  , 0.  , 1.  ],
       [0.  , 1.  , 0.  , 0.  ],
       [1.  , 0.  , 0.  , 0.  ],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.  , 0.  , 1.  , 0.  ],
       [0.  , 1.  , 0.  , 0.  ],
       [0.25, 0.25, 0.25, 0.25]])

In [72]:
V.reshape(4, 4)

array([[0.06889086, 0.06141454, 0.07440974, 0.0558073 ],
       [0.0918545 , 0.        , 0.1122082 , 0.        ],
       [0.14543633, 0.24749694, 0.29961758, 0.        ],
       [0.        , 0.37993589, 0.63902014, 0.        ]])

In [73]:
policy_iteration(env, gamma=0.9)

(array([[1.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 1.  ],
        [1.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 1.  ],
        [1.  , 0.  , 0.  , 0.  ],
        [0.25, 0.25, 0.25, 0.25],
        [0.5 , 0.  , 0.5 , 0.  ],
        [0.25, 0.25, 0.25, 0.25],
        [0.  , 0.  , 0.  , 1.  ],
        [0.  , 1.  , 0.  , 0.  ],
        [1.  , 0.  , 0.  , 0.  ],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.  , 0.  , 1.  , 0.  ],
        [0.  , 1.  , 0.  , 0.  ],
        [0.25, 0.25, 0.25, 0.25]]),
 array([0.06889086, 0.06141454, 0.07440974, 0.0558073 , 0.0918545 ,
        0.        , 0.1122082 , 0.        , 0.14543633, 0.24749694,
        0.29961758, 0.        , 0.        , 0.37993589, 0.63902014,
        0.        ]))

#### Value Iteration

In [74]:
Q

array([[0.01470727, 0.01393801, 0.01393801, 0.01316794],
       [0.00852221, 0.01162969, 0.01086043, 0.01550616],
       [0.02444416, 0.0209521 , 0.02405958, 0.01435233],
       [0.01047585, 0.01047585, 0.00698379, 0.01396775],
       [0.02166341, 0.01701767, 0.0162476 , 0.01006154],
       [0.        , 0.        , 0.        , 0.        ],
       [0.05433495, 0.04735099, 0.05433495, 0.00698396],
       [0.        , 0.        , 0.        , 0.        ],
       [0.01701767, 0.04099176, 0.03480569, 0.04640756],
       [0.0702086 , 0.11755959, 0.10595772, 0.05895286],
       [0.18940397, 0.17582024, 0.16001408, 0.04297362],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.08799662, 0.20503708, 0.23442697, 0.17582024],
       [0.25238807, 0.53837042, 0.52711467, 0.43929106],
       [0.        , 0.        , 0.        , 0.        ]])

###  Value Iteration
<img src='images/value_iteration.png'>

In [75]:
def value_iteration(env, gamma=1.0, theta=1e-8):
    V = np.zeros(env.nS)
    
    while True:
        delta = 0 
        Q = action_value_function(env, V, gamma)
        for s in range(env.nS):
            v_s = V[s]
            V[s] = max(Q[s])
            delta = max(delta, abs(v_s - V[s]))
        if delta<theta:
            break
        
    policy = policy_improvement(env, V, gamma)
    
    return policy, V.reshape(4, 4)

In [76]:
value_iteration(env, gamma=0.9)

(array([[1.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 1.  ],
        [1.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 1.  ],
        [1.  , 0.  , 0.  , 0.  ],
        [0.25, 0.25, 0.25, 0.25],
        [0.5 , 0.  , 0.5 , 0.  ],
        [0.25, 0.25, 0.25, 0.25],
        [0.  , 0.  , 0.  , 1.  ],
        [0.  , 1.  , 0.  , 0.  ],
        [1.  , 0.  , 0.  , 0.  ],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.  , 0.  , 1.  , 0.  ],
        [0.  , 1.  , 0.  , 0.  ],
        [0.25, 0.25, 0.25, 0.25]]),
 array([[0.06889084, 0.06141452, 0.07440972, 0.05580728],
        [0.09185448, 0.        , 0.11220819, 0.        ],
        [0.14543631, 0.24749692, 0.29961757, 0.        ],
        [0.        , 0.37993588, 0.63902014, 0.        ]]))